## Chapter 6. Object References, Mutability, and Recycling

### Variables Are Not Boxes

In [1]:
# Example 6-1. Variables a and b hold references to the same list, not copies of the list

a = [1, 2, 3]
b = a
a.append(4)
b

b.append(5)
a

[1, 2, 3, 4, 5]

In [2]:
# Example 6-2. Variables are assigned to objects only after the objects are created
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))

x = Gizmo()
y = Gizmo() * 10  # y was never created due to the exception on the multiplication
                  # But, the Gizmo is instantiated

Gizmo id: 4424538048
Gizmo id: 4424538672


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

### Identity, Equality, and Alias

In [3]:
# Example 6-3. charles and lewis refer to the same object

charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
print(lewis is charles)
print(id(charles), id(lewis))  # same id
lewis['balance'] = 950
print(charles)

True
4579319680 4579319680
{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}


In [4]:
# Example 6-4 implements and tests the alex object depicted in Figure 8-3

alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}  # a replica of charles

print(alex == charles)
print(alex is not charles)

True
True


#### The Relative Immutability of Tuples

In [5]:
# Example 6-5. t1 and t2 initially compare equal, but changing a mutable item inside tuple t1 makes it different

t1 = (1, 2, [30, 40])  # t1 is immutable, but t1[-1] is mutable.
t2 = (1, 2, [30, 40])
print(t1 == t2)
print(id(t1[-1]))
t1[-1].append(99)
print(t1)
print(id(t1[-1]))  # the ID is not changed.
print(t1 == t2)

True
4579307072
(1, 2, [30, 40, 99])
4579307072
False


### Copies Are Shallow by Default

In [7]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)  # shallow copy
print(l2)
print(l2 == l1)  # True
print(l2 is l1)  # False
print(l2[1] is l1[1])
print(l2[2] is l1[2])
print()

# another example
l3 = l1[:]  # Make a another shallow copy
print(l3)
print(l3 == l1)
print(l3 is l1)
print(l3[1] is l1[1])
print(l3[2] is l1[2])

[3, [55, 44], (7, 8, 9)]
True
False
True
True

[3, [55, 44], (7, 8, 9)]
True
False
True
True


In [8]:
# Example 6-6. Making a shallow copy of a list containing another list; 
# copy and paste this code to see it animated at the Online Python Tutor
# Page 210 ~ 211

l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)  # l2 is a shallow copy of l1
l1.append(100)  # Appending 100 to l1 has no effect on l2.
l1[1].remove(55)  # Removing 55 affects both l1 and l2

print('l1:', l1)
print('l2:', l2)

l2[1] += [33, 22]  # affect both
l2[2] += (10, 11)  # affect l2 only since it creates new tuple
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


#### Deep and Shallow Copies of Arbitrary Objects

In [10]:
# Example 6-8. Bus picks up and drops off passengers

class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)  # make a copy

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [12]:
# Example 6-9. Effects of using copy versus deepcopy

import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
print(id(bus1), id(bus2), id(bus3))

bus1.drop('Bill')
print(bus2.passengers)  # 'Bill' is also missing from bus2
print(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))  # bus1.passengers is bus2.passengers
print(bus3.passengers)


2465177228656 2465177224384 2465177237872
['Alice', 'Claire', 'David']
2465203732096 2465203732096 2465203718144
['Alice', 'Bill', 'Claire', 'David']


In [18]:
# Example 6-10. Cyclic references: b refers to a, and then is appended to a; deepcopy still manages to copy a

a = [10, 20]
b = [a, 30]
a.append(b)
print(a)
print(a[2])
print(a[2][0])  # same as a
print(a[2][0][2])

from copy import deepcopy
c = deepcopy(a)
c

[10, 20, [[...], 30]]
[[10, 20, [...]], 30]
[10, 20, [[...], 30]]
[[10, 20, [...]], 30]


[10, 20, [[...], 30]]

### Function Parameters as References

In [19]:
# Example 6-11. A function may change any mutable object it receives

def f(a, b):
    a += b
    return a

x = 1
y = 2
f(x, y)
print(x, y)

a = [1, 2]
b = [3, 4]
f(a, b)
print(a, b)  # a is changed

t = (10, 20)
u = (30, 40)
f(t, u)
print(t, u)

1 2
[1, 2, 3, 4] [3, 4]
(10, 20) (30, 40)


#### Mutable Types as Parameter Defaults: Bad Idea

In [2]:
# Example 6-12. A simple class to illustrate the danger of a mutable default

class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [3]:
bus1 = HauntedBus(['Alice', 'Bill'])
print(bus1.passengers)
bus1.pick('Charlie')
bus1.drop('Alice')
print(bus1.passengers)  # Works well
print()

bus2 = HauntedBus()
bus2.pick('Carrie')
print(bus2.passengers)
print()

bus3 = HauntedBus()
print(bus3.passengers)  # The default is no longer empty!
bus3.pick('Dave')
print(bus2.passengers)
print()

print(bus2.passengers is bus3.passengers)
print(bus1.passengers)  # bus1.passengers is a distinct list.
print()

print((dir(HauntedBus.__init__)))
print(HauntedBus.__init__.__defaults__)  # it is not empty!
print(HauntedBus.__init__.__defaults__[0] is bus2.passengers)
print(HauntedBus.__init__.__defaults__[0] is bus3.passengers)

['Alice', 'Bill']
['Bill', 'Charlie']

['Carrie']

['Carrie']
['Carrie', 'Dave']

True
['Bill', 'Charlie']

['__annotations__', '__builtins__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
(['Carrie', 'Dave'],)
True
True


#### Defensive Programming with Mutable Parameters

In [27]:
# Example 6-15. A simple class to show the perils of mutating received arguments

class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):  # None is commonly used for the mutable default
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers  # The bus is aliasing the list passed to the constructor
                                          

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [28]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team  # A bus drops Tina and Pat from the team?

['Sue', 'Maya', 'Diana']

In [6]:
# Example 6-15A. A simple class to show the perils of mutating received arguments

class TwilightBus_Fixed:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)  # fixed the aliasing problem
                                          

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [8]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus_Fixed(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
print(bus.passengers)
basketball_team

['Sue', 'Maya', 'Diana']


['Sue', 'Tina', 'Maya', 'Diana', 'Pat']

### del and Garbage Collection

In [1]:
a = [1, 2]
b = a
del a
print(b)  # del a does not affect [1, 2] since b still points to it
b = [3]  # The last remaining reference to [1, 2] is removed. The garbage collector can discard that object

[1, 2]


In [2]:
# Example 6-16. Watching the end of an object when no more references point to it

import weakref

s1 = {1, 2, 3}
s2 = s1

def bye():
    print('Gone with the wind...')

ender = weakref.finalize(s1, bye)  # Register the bye callback on the object referred by s1.
print(ender.alive)
del s1  # del did not delete the object, just the s1 reference to it
print(ender.alive)
s2 = 'spam'
print(ender.alive)

True
True
Gone with the wind...
False


### Tricks Python Plays with Immutables

In [38]:
# Example 6-17. A tuple built from another is actually the same exact tuple

t1 = (1, 2, 3)
t2 = tuple(t1)
print(t2 is t1)

t3 = t1[:]
print(t3 is t1)

True
True


In [39]:
# Example 6-18. String literals may create shared objects

t1 = (1, 2, 3)
t3 = (1, 2, 3) 
print(t3 is t1)

s1 = 'ABC'
s2 = 'ABC'
print(s2 is s1)  # This is a result of an optimization technique called 'interning'

False
True
